In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip './drive/MyDrive/Data/archive.zip'

In [ ]:
import warnings  # Import the 'warnings' module for handling warnings
warnings.filterwarnings("ignore")  # Ignore warnings during execution


In [ ]:
!pip install -U -q evaluate transformers datasets>=2.14.5 accelerate>=0.27 2>/dev/null

In [ ]:
!pip install imblearn

In [ ]:
import gc  # Import the 'gc' module for garbage collection
import numpy as np  # Import NumPy for numerical operations
import pandas as pd  # Import Pandas for data manipulation
import itertools
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    f1_score
)

from imblearn.over_sampling import RandomOverSampler # import RandomOverSampler
import accelerate # Import the 'accelerate' module
import evaluate  # Import the 'evaluate' module
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import (  # Import various modules from the Transformers library
    TrainingArguments,  # For training arguments
    Trainer,  # For model training
    ViTImageProcessor,  # For processing image data with ViT models
    ViTForImageClassification,  # ViT model for image classification
    DefaultDataCollator  # For collating data in the default way
)
import torch  # Import PyTorch for deep learning
from torch.utils.data import DataLoader  # For creating data loaders
from torchvision.transforms import (  # Import image transformation functions
    CenterCrop,  # Center crop an image
    Compose,  # Compose multiple image transformations
    Normalize,  # Normalize image pixel values
    RandomRotation,  # Apply random rotation to images
    RandomResizedCrop,  # Crop and resize images randomly
    RandomHorizontalFlip,  # Apply random horizontal flip
    RandomAdjustSharpness,  # Adjust sharpness randomly
    Resize,  # Resize images
    ToTensor  # Convert images to PyTorch tensors
)
# Import the necessary module from the Python Imaging Library (PIL).
from PIL import Image, ImageFile

from transformers import ViTImageProcessor
from torchvision.transforms import (Compose, Resize, RandomRotation,
                                    RandomAdjustSharpness, Lambda,
                                    ToTensor, Normalize)

In [ ]:

# Enable the option to load truncated images.
# This setting allows the PIL library to attempt loading images even if they are corrupted or incomplete.
ImageFile.LOAD_TRUNCATED_IMAGES = True
# use https://huggingface.co/docs/datasets/image_load for reference
# Import necessary libraries
image_dict = {}



In [ ]:

# Define the list of file names
from pathlib import Path
from tqdm import tqdm
import os
# Initialize empty lists to store file names and labels
file_names = []
labels = []
# Iterate through all image files in the specified directory
for file in sorted((Path('./dataset').glob('*/*.*'))):
    # check number of such files in a directory
    sample_dir = '/'.join(str(file).split('/')[:-1])+'/'
    file_names.append(str(file))  # Add the file path to the list
    label = str(file).split('/')[-2]  # Extract the label from the file path
    labels.append(label)  # Add the label to the list
# Print the total number of file names and labels
print(len(file_names), len(labels), len(set(labels)))

77382 77382 100


In [ ]:
# Create a pandas dataframe from the collected file names and labels
df = pd.DataFrame.from_dict({"image": file_names, "label": labels})
# random oversampling of all minority classes
y = df[['label']]
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df, y_resampled = ros.fit_resample(df, y)
del y
df['label'] = y_resampled
del y_resampled
gc.collect()

0

In [ ]:
# Create a list of unique labels by converting 'labels' to a set and then back to a list
labels_list = ['Urnula craterium', 'Leccinum albostipitatum', 'Lactarius deliciosus',
               'Clitocybe nebularis', 'Hypholoma fasciculare', 'Lactarius torminosus',
               'Lycoperdon perlatum', 'Verpa bohemica', 'Schizophyllum commune',
               'Leccinum aurantiacum', 'Phellinus igniarius', 'Suillus luteus',
               'Coltricia perennis', 'Cetraria islandica', 'Amanita muscaria',
               'Pholiota aurivella', 'Trichaptum biforme', 'Artomyces pyxidatus',
               'Calocera viscosa', 'Sarcosoma globosum', 'Evernia prunastri',
               'Laetiporus sulphureus', 'Lobaria pulmonaria', 'Bjerkandera adusta',
               'Vulpicida pinastri', 'Imleria badia', 'Evernia mesomorpha',
               'Physcia adscendens', 'Coprinellus micaceus', 'Armillaria borealis',
               'Trametes ochracea', 'Cantharellus cibarius', 'Pseudevernia furfuracea',
               'Tremella mesenterica', 'Gyromitra infula', 'Leccinum versipelle',
               'Mutinus ravenelii', 'Pholiota squarrosa', 'Amanita rubescens',
               'Amanita pantherina', 'Sarcoscypha austriaca', 'Boletus edulis',
               'Coprinus comatus', 'Merulius tremellosus', 'Stropharia aeruginosa',
               'Cladonia fimbriata', 'Suillus grevillei', 'Apioperdon pyriforme',
               'Cerioporus squamosus', 'Leccinum scabrum', 'Rhytisma acerinum',
               'Hypholoma lateritium', 'Flammulina velutipes', 'Tricholomopsis rutilans',
               'Coprinopsis atramentaria', 'Trametes versicolor', 'Graphis scripta',
               'Ganoderma applanatum', 'Phellinus tremulae', 'Peltigera aphthosa',
               'Parmelia sulcata', 'Fomitopsis betulina', 'Pleurotus pulmonarius',
               'Fomitopsis pinicola', 'Daedaleopsis confragosa', 'Hericium coralloides',
               'Trametes hirsuta', 'Coprinellus disseminatus', 'Kuehneromyces mutabilis',
               'Pleurotus ostreatus', 'Phlebia radiata', 'Boletus reticulatus',
               'Phallus impudicus', 'Macrolepiota procera', 'Fomes fomentarius',
               'Suillus granulatus', 'Gyromitra esculenta', 'Xanthoria parietina',
               'Nectria cinnabarina', 'Sarcomyxa serotina', 'Inonotus obliquus',
               'Panellus stipticus', 'Hypogymnia physodes', 'Hygrophoropsis aurantiaca',
               'Cladonia rangiferina', 'Platismatia glauca', 'Calycina citrina',
               'Cladonia stellaris', 'Amanita citrina', 'Lepista nuda', 'Gyromitra gigas',
               'Crucibulum laeve', 'Daedaleopsis tricolor', 'Stereum hirsutum',
               'Paxillus involutus', 'Lactarius turpis', 'Chlorociboria aeruginascens',
               'Chondrostereum purpureum', 'Phaeophyscia orbicularis', 'Peltigera praetextata'] #list(set(labels))

# Initialize empty dictionaries to map labels to IDs and vice versa
label2id, id2label = dict(), dict()

In [ ]:
# Iterate over the unique labels and assign each label an ID, and vice versa
for i, label in enumerate(labels_list):
    label2id[label] = i  # Map the label to its corresponding ID
    id2label[i] = label

# Print the resulting dictionaries for reference
print("Mapping of IDs to Labels:", id2label, '\n')
print("Mapping of Labels to IDs:", label2id)
# Creating classlabels to match labels to IDs
ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)


Mapping of IDs to Labels: {0: 'Urnula craterium', 1: 'Leccinum albostipitatum', 2: 'Lactarius deliciosus', 3: 'Clitocybe nebularis', 4: 'Hypholoma fasciculare', 5: 'Lactarius torminosus', 6: 'Lycoperdon perlatum', 7: 'Verpa bohemica', 8: 'Schizophyllum commune', 9: 'Leccinum aurantiacum', 10: 'Phellinus igniarius', 11: 'Suillus luteus', 12: 'Coltricia perennis', 13: 'Cetraria islandica', 14: 'Amanita muscaria', 15: 'Pholiota aurivella', 16: 'Trichaptum biforme', 17: 'Artomyces pyxidatus', 18: 'Calocera viscosa', 19: 'Sarcosoma globosum', 20: 'Evernia prunastri', 21: 'Laetiporus sulphureus', 22: 'Lobaria pulmonaria', 23: 'Bjerkandera adusta', 24: 'Vulpicida pinastri', 25: 'Imleria badia', 26: 'Evernia mesomorpha', 27: 'Physcia adscendens', 28: 'Coprinellus micaceus', 29: 'Armillaria borealis', 30: 'Trametes ochracea', 31: 'Cantharellus cibarius', 32: 'Pseudevernia furfuracea', 33: 'Tremella mesenterica', 34: 'Gyromitra infula', 35: 'Leccinum versipelle', 36: 'Mutinus ravenelii', 37: 'Ph

In [ ]:

# Mapping labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example


In [ ]:
# Load dataset from dataframe
dataset = Dataset.from_pandas(df)

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)

# Splitting the dataset into training and testing sets using an 60-40 split ratio.
dataset = dataset.train_test_split(test_size=0.3, shuffle=True, stratify_by_column="label")

# Extracting the training data from the split dataset.
train_data = dataset['train']

# Extracting the testing data from the split dataset.
test_data = dataset['test']

# Define the pre-trained ViT model string
model_str = "dima806/mushrooms_image_detection" #'google/vit-base-patch16-224-in21k'

Map:   0%|          | 0/583700 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/583700 [00:00<?, ? examples/s]

In [ ]:
from transformers import ViTImageProcessor
from torchvision.transforms import (Compose, Resize, RandomRotation,
                                    RandomAdjustSharpness, Lambda,
                                    ToTensor, Normalize)
# Create a processor for ViT model input from the pre-trained model
processor = ViTImageProcessor.from_pretrained(model_str)

# Retrieve the image mean and standard deviation used for normalization
image_mean, image_std = processor.image_mean, processor.image_std

# Get the size (height) of the ViT model's input images
size = processor.size["height"]
print("Size: ", size)

# Define a normalization transformation for the input images
normalize = Normalize(mean=image_mean, std=image_std)

# Define a set of transformations for training data
_train_transforms = Compose(
    [
        Resize((size, size)),             # Resize images to the ViT model's input size
        RandomRotation(90),               # Apply random rotation up to 90 degrees
        RandomAdjustSharpness(2),         # Adjust sharpness randomly
        Lambda(lambda x: x.convert("L")), # Convert images to grayscale
        ToTensor(),                       # Convert images to tensors
        normalize                         # Normalize images using mean and std
    ]
)

# Define a set of transformations for validation data
_val_transforms = Compose(
    [
        Resize((size, size)),             # Resize images to the ViT model's input size
        Lambda(lambda x: x.convert("L")), # Convert images to grayscale
        ToTensor(),                       # Convert images to tensors
        normalize                         # Normalize images using mean and std
    ]
)


preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

Size:  224


In [ ]:

# Define a function to apply training transformations to a batch of examples
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(Image.open(image_path).convert("RGB")) for image_path in examples['image']]
    return examples

# Define a function to apply validation transformations to a batch of examples
def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(Image.open(image_path).convert("RGB")) for image_path in examples['image']]
    return examples

# Set the transforms for the training data
train_data.set_transform(train_transforms)

# Set the transforms for the test/validation data
test_data.set_transform(val_transforms)


In [ ]:

# Define a collate function that prepares batched data for model training.
def collate_fn(examples):
    # Stack the pixel values from individual examples into a single tensor.
    pixel_values = torch.stack([example["pixel_values"] for example in examples])

    # Convert the label strings in examples to corresponding numeric IDs using label2id dictionary.
    labels = torch.tensor([example['label'] for example in examples])

    # Return a dictionary containing the batched pixel values and labels.
    return {"pixel_values": pixel_values, "labels": labels}

# Create a ViTForImageClassification model from a pretrained checkpoint with a specified number of output labels.
model = ViTForImageClassification.from_pretrained(model_str, num_labels=len(labels_list))

# Configure the mapping of class labels to their corresponding indices for later reference.
model.config.id2label = id2label
model.config.label2id = label2id

# Calculate and print the number of trainable parameters in millions for the model.
print(model.num_parameters(only_trainable=True) / 1e6)

# Load the accuracy metric from a module named 'evaluate'
accuracy = evaluate.load("accuracy")

# Define a function 'compute_metrics' to calculate evaluation metrics
def compute_metrics(eval_pred):
    # Extract model predictions from the evaluation prediction object
    predictions = eval_pred.predictions

    # Extract true labels from the evaluation prediction object
    label_ids = eval_pred.label_ids

    # Calculate accuracy using the loaded accuracy metric
    # Convert model predictions to class labels by selecting the class with the highest probability (argmax)
    predicted_labels = predictions.argmax(axis=1)

    # Calculate accuracy score by comparing predicted labels to true labels
    acc_score = accuracy.compute(predictions=predicted_labels, references=label_ids)['accuracy']

    # Return the computed accuracy as a dictionary with the key "accuracy"
    return {
        "accuracy": acc_score
    }


config.json:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/344M [00:00<?, ?B/s]

85.875556


In [ ]:

# Define the name of the evaluation metric to be used during training and evaluation.
metric_name = "accuracy"

# Define the name of the model, which will be used to create a directory for saving model checkpoints and outputs.
model_name = "mushrooms_image_detection"

# Define the number of training epochs for the model.
num_train_epochs = 1


In [ ]:

# Create an instance of TrainingArguments to configure training settings.
args = TrainingArguments(
    # Specify the directory where model checkpoints and outputs will be saved.
    output_dir=model_name,

    # Specify the directory where training logs will be stored.
    logging_dir='./logs',

    # Define the evaluation strategy, which is performed at the end of each epoch.
    evaluation_strategy="epoch",

    # Set the learning rate for the optimizer.
    learning_rate=2e-7,

    # Define the batch size for training on each device.
    per_device_train_batch_size=32,

    # Define the batch size for evaluation on each device.
    per_device_eval_batch_size=8,

    # Specify the total number of training epochs.
    num_train_epochs=num_train_epochs,

    # Apply weight decay to prevent overfitting.
    weight_decay=0.02,

    # Set the number of warm-up steps for the learning rate scheduler.
    warmup_steps=50,

    # Disable the removal of unused columns from the dataset.
    remove_unused_columns=False,

    # Define the strategy for saving model checkpoints (per epoch in this case).
    save_strategy='epoch',

    # Load the best model at the end of training.
    load_best_model_at_end=True,

    # Limit the total number of saved checkpoints to save space.
    save_total_limit=1,

    # Specify that training progress should not be reported.
    report_to="none"
)



In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

trainer.evaluate()


RuntimeError: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]

In [ ]:
# Enable the option to load truncated images.
# This setting allows the PIL library to attempt loading images even if they are corrupted or incomplete.
ImageFile.LOAD_TRUNCATED_IMAGES = True
# use https://huggingface.co/docs/datasets/image_load for reference

# Import necessary libraries
image_dict = {}

# Define the list of file names
from pathlib import Path
from tqdm import tqdm
import os
# Initialize empty lists to store file names and labels
file_names = []
labels = []
# Iterate through all image files in the specified directory
for file in sorted((Path('./dataset').glob('*/*.*'))):
    # check number of such files in a directory
    sample_dir = '/'.join(str(file).split('/')[:-1])+'/'
    file_names.append(str(file))  # Add the file path to the list
    label = str(file).split('/')[-2]  # Extract the label from the file path
    labels.append(label)  # Add the label to the list

# Print the total number of file names and labels
print(len(file_names), len(labels), len(set(labels)))

# Create a pandas dataframe from the collected file names and labels
df = pd.DataFrame.from_dict({"image": file_names, "label": labels})
# random oversampling of all minority classes
y = df[['label']]
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df, y_resampled = ros.fit_resample(df, y)
del y
df['label'] = y_resampled
del y_resampled
gc.collect()

print(df.shape)
# Create a list of unique labels by converting 'labels' to a set and then back to a list
# Create a list of unique labels by converting 'labels' to a set and then back to a list
labels_list = ['Urnula craterium', 'Leccinum albostipitatum', 'Lactarius deliciosus',
               'Clitocybe nebularis', 'Hypholoma fasciculare', 'Lactarius torminosus',
               'Lycoperdon perlatum', 'Verpa bohemica', 'Schizophyllum commune',
               'Leccinum aurantiacum', 'Phellinus igniarius', 'Suillus luteus',
               'Coltricia perennis', 'Cetraria islandica', 'Amanita muscaria',
               'Pholiota aurivella', 'Trichaptum biforme', 'Artomyces pyxidatus',
               'Calocera viscosa', 'Sarcosoma globosum', 'Evernia prunastri',
               'Laetiporus sulphureus', 'Lobaria pulmonaria', 'Bjerkandera adusta',
               'Vulpicida pinastri', 'Imleria badia', 'Evernia mesomorpha',
               'Physcia adscendens', 'Coprinellus micaceus', 'Armillaria borealis',
               'Trametes ochracea', 'Cantharellus cibarius', 'Pseudevernia furfuracea',
               'Tremella mesenterica', 'Gyromitra infula', 'Leccinum versipelle',
               'Mutinus ravenelii', 'Pholiota squarrosa', 'Amanita rubescens',
               'Amanita pantherina', 'Sarcoscypha austriaca', 'Boletus edulis',
               'Coprinus comatus', 'Merulius tremellosus', 'Stropharia aeruginosa',
               'Cladonia fimbriata', 'Suillus grevillei', 'Apioperdon pyriforme',
               'Cerioporus squamosus', 'Leccinum scabrum', 'Rhytisma acerinum',
               'Hypholoma lateritium', 'Flammulina velutipes', 'Tricholomopsis rutilans',
               'Coprinopsis atramentaria', 'Trametes versicolor', 'Graphis scripta',
               'Ganoderma applanatum', 'Phellinus tremulae', 'Peltigera aphthosa',
               'Parmelia sulcata', 'Fomitopsis betulina', 'Pleurotus pulmonarius',
               'Fomitopsis pinicola', 'Daedaleopsis confragosa', 'Hericium coralloides',
               'Trametes hirsuta', 'Coprinellus disseminatus', 'Kuehneromyces mutabilis',
               'Pleurotus ostreatus', 'Phlebia radiata', 'Boletus reticulatus',
               'Phallus impudicus', 'Macrolepiota procera', 'Fomes fomentarius',
               'Suillus granulatus', 'Gyromitra esculenta', 'Xanthoria parietina',
               'Nectria cinnabarina', 'Sarcomyxa serotina', 'Inonotus obliquus',
               'Panellus stipticus', 'Hypogymnia physodes', 'Hygrophoropsis aurantiaca',
               'Cladonia rangiferina', 'Platismatia glauca', 'Calycina citrina',
               'Cladonia stellaris', 'Amanita citrina', 'Lepista nuda', 'Gyromitra gigas',
               'Crucibulum laeve', 'Daedaleopsis tricolor', 'Stereum hirsutum',
               'Paxillus involutus', 'Lactarius turpis', 'Chlorociboria aeruginascens',
               'Chondrostereum purpureum', 'Phaeophyscia orbicularis', 'Peltigera praetextata'] #list(set(labels))

# Initialize empty dictionaries to map labels to IDs and vice versa
label2id, id2label = dict(), dict()

# Iterate over the unique labels and assign each label an ID, and vice versa
for i, label in enumerate(labels_list):
    label2id[label] = i  # Map the label to its corresponding ID
    id2label[i] = label  # Map the ID to its corresponding label

# Print the resulting dictionaries for reference
print("Mapping of IDs to Labels:", id2label, '\n')
print("Mapping of Labels to IDs:", label2id)

# Creating classlabels to match labels to IDs
ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)

# Mapping labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

# Load dataset from dataframe
dataset = Dataset.from_pandas(df)

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)

# Splitting the dataset into training and testing sets using an 60-40 split ratio.
dataset = dataset.train_test_split(test_size=0.3, shuffle=True, stratify_by_column="label")

# Extracting the training data from the split dataset.
train_data = dataset['train']

# Extracting the testing data from the split dataset.
test_data = dataset['test']

# Define the pre-trained ViT model string
model_str = "dima806/mushrooms_image_detection" #'google/vit-base-patch16-224-in21k'

# Define mean and std for single-channel grayscale images.
grayscale_mean = [0.5]  # Example mean
grayscale_std = [0.5]   # Example std

# Define a normalization transformation for the grayscale input images
normalize = Normalize(mean=grayscale_mean, std=grayscale_std)

# Define a set of transformations for training data
_train_transforms = Compose(
    [
        Resize((size, size)),             # Resize images to the ViT model's input size
        RandomRotation(90),               # Apply random rotation up to 90 degrees
        RandomAdjustSharpness(2),         # Adjust sharpness randomly
        Lambda(lambda x: x.convert("L")), # Convert images to grayscale
        ToTensor(),                       # Convert images to tensors
        normalize                         # Normalize images using mean and std
    ]
)

# Define a set of transformations for validation data
_val_transforms = Compose(
    [
        Resize((size, size)),             # Resize images to the ViT model's input size
        Lambda(lambda x: x.convert("L")), # Convert images to grayscale
        ToTensor(),                       # Convert images to tensors
        normalize                         # Normalize images using mean and std
    ]
)

# Define a function to apply training transformations to a batch of examples
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(Image.open(image_path)) for image_path in examples['image']]
    return examples

# Define a function to apply validation transformations to a batch of examples
def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(Image.open(image_path)) for image_path in examples['image']]
    return examples

# Set the transforms for the training data
train_data.set_transform(train_transforms)

# Set the transforms for the test/validation data
test_data.set_transform(val_transforms)

# Define a collate function that prepares batched data for model training.
def collate_fn(examples):
    # Stack the pixel values from individual examples into a single tensor.
    pixel_values = torch.stack([example["pixel_values"] for example in examples])

    # Convert the label strings in examples to corresponding numeric IDs using label2id dictionary.
    labels = torch.tensor([example['label'] for example in examples])

    # Return a dictionary containing the batched pixel values and labels.
    return {"pixel_values": pixel_values, "labels": labels}

# Create a ViTForImageClassification model from a pretrained checkpoint with a specified number of output labels.
model = ViTForImageClassification.from_pretrained(model_str, num_labels=len(labels_list))

# Configure the mapping of class labels to their corresponding indices for later reference.
model.config.id2label = id2label
model.config.label2id = label2id

# Calculate and print the number of trainable parameters in millions for the model.
print(model.num_parameters(only_trainable=True) / 1e6)

# Load the accuracy metric from a module named 'evaluate'
accuracy = evaluate.load("accuracy")

# Define a function 'compute_metrics' to calculate evaluation metrics
def compute_metrics(eval_pred):
    # Extract model predictions from the evaluation prediction object
    predictions = eval_pred.predictions

    # Extract true labels from the evaluation prediction object
    label_ids = eval_pred.label_ids

    # Calculate accuracy using the loaded accuracy metric
    # Convert model predictions to class labels by selecting the class with the highest probability (argmax)
    predicted_labels = predictions.argmax(axis=1)

    # Calculate accuracy score by comparing predicted labels to true labels
    acc_score = accuracy.compute(predictions=predicted_labels, references=label_ids)['accuracy']

    # Return the computed accuracy as a dictionary with the key "accuracy"
    return {
        "accuracy": acc_score
    }

# Define the name of the evaluation metric to be used during training and evaluation.
metric_name = "accuracy"

# Define the name of the model, which will be used to create a directory for saving model checkpoints and outputs.
model_name = "mushrooms_image_detection"

# Define the number of training epochs for the model.
num_train_epochs = 1

# Create an instance of TrainingArguments to configure training settings.
args = TrainingArguments(
    # Specify the directory where model checkpoints and outputs will be saved.
    output_dir=model_name,

    # Specify the directory where training logs will be stored.
    logging_dir='./logs',

    # Define the evaluation strategy, which is performed at the end of each epoch.
    evaluation_strategy="epoch",

    # Set the learning rate for the optimizer.
    learning_rate=2e-7,

    # Define the batch size for training on each device.
    per_device_train_batch_size=32,

    # Define the batch size for evaluation on each device.
    per_device_eval_batch_size=8,

    # Specify the total number of training epochs.
    num_train_epochs=num_train_epochs,

    # Apply weight decay to prevent overfitting.
    weight_decay=0.02,

    # Set the number of warm-up steps for the learning rate scheduler.
    warmup_steps=50,

    # Disable the removal of unused columns from the dataset.
    remove_unused_columns=False,

    # Define the strategy for saving model checkpoints (per epoch in this case).
    save_strategy='epoch',

    # Load the best model at the end of training.
    load_best_model_at_end=True,

    # Limit the total number of saved checkpoints to save space.
    save_total_limit=1,

    # Specify that training progress should not be reported.
    report_to="none"
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

trainer.evaluate()
outputs = trainer.predict(test_data)

print(outputs.metrics)

##Yang ini Yang Bener

In [ ]:
#Yang ini yang bener

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Initialize lists for file names and labels
file_names = []
labels = []
for file in sorted((Path('./dataset').glob('*/*.*'))):
    sample_dir = '/'.join(str(file).split('/')[:-1])+'/'
    file_names.append(str(file))
    label = str(file).split('/')[-2]
    labels.append(label)

print(len(file_names), len(labels), len(set(labels)))
df = pd.DataFrame.from_dict({"image": file_names, "label": labels})

# Oversample minority classes
y = df[['label']]
df = df.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
df, y_resampled = ros.fit_resample(df, y)
df['label'] = y_resampled
gc.collect()

print(df.shape)
labels_list = sorted(set(labels))
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for i, label in enumerate(labels_list)}

print("Mapping of IDs to Labels:", id2label, '\n')
print("Mapping of Labels to IDs:", label2id)

ClassLabels = ClassLabel(num_classes=len(labels_list), names=labels_list)

def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

dataset = Dataset.from_pandas(df)
dataset = dataset.map(map_label2id, batched=True)
dataset = dataset.cast_column('label', ClassLabels)
dataset = dataset.train_test_split(test_size=0.3, shuffle=True, stratify_by_column="label")

train_data = dataset['train']
test_data = dataset['test']
model_str = "dima806/mushrooms_image_detection"

processor = ViTImageProcessor.from_pretrained(model_str)
image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]
print("Size: ", size)
normalize = Normalize(mean=image_mean, std=image_std)

# Transformations, converting grayscale back to RGB by replication of channels
_train_transforms = Compose([
    Resize((size, size)),
    RandomRotation(90),
    RandomAdjustSharpness(2),
    Lambda(lambda x: x.convert("L").convert("RGB")),  # convert to grayscale and then RGB
    ToTensor(),
    normalize
])

_val_transforms = Compose([
    Resize((size, size)),
    Lambda(lambda x: x.convert("L").convert("RGB")),  # convert to grayscale and then RGB
    ToTensor(),
    normalize
])

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(Image.open(image_path)) for image_path in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(Image.open(image_path)) for image_path in examples['image']]
    return examples

train_data.set_transform(train_transforms)
test_data.set_transform(val_transforms)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example['label'] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

model = ViTForImageClassification.from_pretrained(model_str, num_labels=len(labels_list))
model.config.id2label = id2label
model.config.label2id = label2id
print(model.num_parameters(only_trainable=True) / 1e6)

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    label_ids = eval_pred.label_ids
    predicted_labels = predictions.argmax(axis=1)
    acc_score = accuracy.compute(predictions=predicted_labels, references=label_ids)['accuracy']
    return {"accuracy": acc_score}

metric_name = "accuracy"
model_name = "mushrooms_image_detection"
num_train_epochs = 1

args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    learning_rate=2e-7,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    warmup_steps=50,
    remove_unused_columns=False,
    save_strategy='epoch',
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none"
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

trainer.evaluate()
outputs = trainer.predict(test_data)
print(outputs.metrics)

In [ ]:
outputs = trainer.predict(test_data)

print(outputs.metrics)


In [ ]:
y_true = outputs.label_ids

y_pred = outputs.predictions.argmax(1)

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='macro')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print()
print("Classification report:")
print()
print(classification_report(y_true, y_pred, target_names=labels_list, digits=4))

In [ ]:
trainer.save_model(output_dir='./drive/MyDrive/Data')

In [ ]:
trainer.tokenizer.save_pretrained("./drive/MyDrive/Data")